In [2]:
import os


In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Courses\\Udemy\\Complete MLOps Bootcamp\\Projects\\7_End to End Projects\\end-to-end-data-science-project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainConfig:
    root_dir: Path
    train_data_path: Path
    validation_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [7]:
from src.project.constants import *
from src.project.utils.common import read_yaml, create_directories
from src.project import logger

In [8]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                  params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)   

        create_directories([self.config.artifacts_root])
    
    def get_model_train_config(self)-> ModelTrainConfig:
        config=self.config.model_train
        params = self.params.ElasticNet
        target = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])

        model_train_config = ModelTrainConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            validation_data_path=config.validation_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = target.name
            
        )
        return model_train_config

In [9]:
import pandas as pd
from sklearn.linear_model import ElasticNet
import joblib

In [11]:
class ModelTrain:
    def __init__(self, config: ModelTrainConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        validation_data = pd.read_csv(self.config.validation_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        validation_x = validation_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        
        
        train_y = train_data[[self.config.target_column]]
        validation_y = validation_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        model = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        model.fit(train_x, train_y)

        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))   

In [12]:
try:
    config = ConfigurationManager()
    model_train_config = config.get_model_train_config()
    model_train = ModelTrain(config=model_train_config)
    model_train.train()
except Exception as e:
    raise e

[2025-09-06 15:45:30,556: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-06 15:45:30,562: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-06 15:45:30,586: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-06 15:45:30,615: INFO: common: created directory at: artifacts]
[2025-09-06 15:45:30,618: INFO: common: created directory at: artifacts/model_train]
